In [ ]:
import os
import sys
sys.path.append(os.path.abspath("..")) 
from dynamic_model.Debt import RiskFree, RiskDebt
from dynamic_model.Tetworks import BellmanNet_RiskyFree, BellmanNet_RiskDebt
from dynamic_modelTrainer import BellmanTrainer
import numpy as np
import tensorflow as tf

2025-12-31 04:04:16.187488: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-31 04:04:17.540931: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
model  = RiskFree()
net =  BellmanNet_RiskyFree(model)

trainer = BellmanTrainer(
    model=model,
    net=net,
    batch_size=64,
    lr= 1e-4,
)

2025-12-31 04:04:22.188561: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14633 MB memory:  -> device: 0, name: Tesla V100-SXM2-16GB, pci bus id: 0000:61:00.0, compute capability: 7.0
2025-12-31 04:04:22.189299: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 14633 MB memory:  -> device: 1, name: Tesla V100-SXM2-16GB, pci bus id: 0000:62:00.0, compute capability: 7.0
2025-12-31 04:04:22.189971: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 14634 MB memory:  -> device: 2, name: Tesla V100-SXM2-16GB, pci bus id: 0000:89:00.0, compute capability: 7.0
2025-12-31 04:04:22.190546: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 14634 MB memory:  -> device: 3, name: Tesla V100-SXM2-16GB, pci bus id

In [3]:
trainer.fit(training_steps = 2000,  display_step= 500, early_stop = False)
           

Training progress:   0%|                                                                              | 0/2000 [00:00<?]

Starting training ...


2025-12-31 04:04:22.626414: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8801


Network weights initialized.


2025-12-31 04:04:28.349355: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0xbd133240 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-12-31 04:04:28.349382: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Tesla V100-SXM2-16GB, Compute Capability 7.0
2025-12-31 04:04:28.349389: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (1): Tesla V100-SXM2-16GB, Compute Capability 7.0
2025-12-31 04:04:28.349394: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (2): Tesla V100-SXM2-16GB, Compute Capability 7.0
2025-12-31 04:04:28.349398: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (3): Tesla V100-SXM2-16GB, Compute Capability 7.0
2025-12-31 04:04:28.356958: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-12-31 04:04:2

...

Epoch  1500 | loss_total=3.596898e+02 | loss_V=3.567528e+02 | loss_FOC_K=1.337065e-03 | loss_FB=0.000000e+00
 Eval @ epoch 1500: lifetime reward = 7.022e+02


Training progress: 100%|██████████████████████████████████████████████████████████████████████▊| 1996/2000 [01:14<00:00]

Epoch  2000 | loss_total=3.992848e+02 | loss_V=3.963669e+02 | loss_FOC_K=6.975376e-04 | loss_FB=0.000000e+00
 Eval @ epoch 2000: lifetime reward = 7.022e+02


Training progress: 100%|███████████████████████████████████████████████████████████████████████| 2000/2000 [01:14<00:00]

Training completed.
Best reward: 7.026e+02 @ epoch 10


In [4]:
import tensorflow as tf


def r_tilde_diff(
    r_old, r_new, 
    tol_default=0.0,
    cross_penalty=1.0    # 现在用于显示分类占比，不放大差异
):
    default_old = r_old > 1e7
    default_new = r_new > 1e7

    mask_both_ok = tf.logical_and(~default_old, ~default_new) # No default for both
    mask_cross = tf.not_equal(default_old, default_new) # One default, one not
    mask_both_default = tf.logical_and(default_old, default_new) # Both default


    diff_normal = tf.abs(r_new - r_old)
    diff_cross  = cross_penalty  * tf.ones_like(r_old, dtype=tf.float32) 
    diff_both   = tf.zeros_like(r_old)

    diff_map = (
        tf.where(mask_both_ok, diff_normal, tf.zeros_like(diff_normal)) +
        tf.where(mask_cross, diff_cross, tf.zeros_like(diff_cross)) +
        tf.where(mask_both_default, diff_both, tf.zeros_like(diff_both))
    )

  
    N_total = tf.cast(tf.size(r_old), tf.float32)
    num_ok = tf.reduce_sum(tf.cast(mask_both_ok, tf.float32))
    num_cross = tf.reduce_sum(tf.cast(mask_cross, tf.float32))
    num_default = tf.reduce_sum(tf.cast(mask_both_default, tf.float32))

    ratio_ok = num_ok / N_total * 100
    ratio_cross = num_cross / N_total  * 100
    ratio_default = num_default / N_total * 100

    diff_mean = tf.reduce_mean(diff_map)
        
    tf.print(
        "both_ok: %.2f%%" % ratio_ok,
        "cross: %.2f%%" % ratio_cross,
        "both_default: %.2f%%" % ratio_default
    )
    all_default = tf.equal(num_default, N_total)
    if all_default:
        tf.print(" All states in default, not counted as convergence.")
        diff_mean = tf.constant(1e6, dtype=tf.float32)

    return diff_mean

In [5]:
riskfree_model = model
riskfree_net = net



In [6]:
RiskDebt_model = RiskDebt(prev_net= riskfree_net)
riskdebt_net = BellmanNet_RiskDebt(RiskDebt_model)


In [7]:


batch_size     = 64
learning_rate  = 1e-4
max_outer_iter = 20          
max_inner_iter = 2000        
tol_rdiff      = 0.05     


test_data = RiskDebt_model.sample_state_test(10000) 
K, B, Z = test_data
prev_net = riskfree_net
r_tilde_old = tf.ones((K.shape[0], 1), dtype=tf.float32) * model.r 

for outer in range(max_outer_iter):

    print(f"\n Outer iteration {outer+1}/{max_outer_iter}")

    riskdebt_model = RiskDebt(prev_net=prev_net, alpha=0.1)
    riskdebt_net   = BellmanNet_RiskDebt(riskdebt_model)

    # inner loop trainer
    trainer = BellmanTrainer(
        model=riskdebt_model,
        net=riskdebt_net,
        train_data=None,
        batch_size=batch_size,
        lr=learning_rate
    )
    
    trainer.fit(
        training_steps=max_inner_iter,
        display_step = 500,
        eval=True,
        eval_interval=5,
        early_stop= True,  
        reward_drop_tolerance= 10,
        n_eval_points= 2048
    )

    r_tilde_new = riskdebt_model.get_r_tilde_test(riskdebt_net, test_data)
    Value_new = riskdebt_net.value(K, B, Z)
    
    diff_r_mean = r_tilde_diff(r_tilde_old, r_tilde_new)
    tf.print(f"Mean weighted abs diff of r_tilde (incl. default):", diff_r_mean)
    #print(f"Mean abs diff between r_tilde: {diff_r_mean:.6f}")
    r_tilde_old = r_tilde_new
  
    if diff_r_mean < tol_rdiff:
        print(f" Converged after {outer+1} outer iterations. Stop training.")
        break

    prev_net = riskdebt_net





 Outer iteration 1/20


Training progress:   0%|                                                                              | 0/2000 [00:00<?]

Starting training ...
Network weights initialized.


Training progress:   0%|                                                                              | 0/2000 [00:05<?]

Epoch     1 | loss_total=5.054780e+05 | loss_V=5.054568e+05 | loss_FOC_K=1.123483e+00 | loss_FOC_B=9.980808e-01
 Eval @ epoch 1: lifetime reward = 7.156e+02


Training progress:  11%|███████▋                                                                | 215/2000 [00:14<01:11]

Overfitting detected at epoch 220. Best reward = 1.0336e+03 (epoch 170)


Training progress:  11%|███████▉                                                                | 219/2000 [00:14<01:58]

Training completed.
Best reward: 1.034e+03 @ epoch 170
Training stopped early (overfitting at epoch 220). Restored best model from epoch 170.


both_ok: 100.00% cross: 0.00% both_default: 0.00%
Mean weighted abs diff of r_tilde (incl. default): 1.86264515e-08
 Converged after 1 outer iterations. Stop training.
